In [2]:
from google.cloud import bigquery
import pandas as pd

In [8]:
client = bigquery.Client()
outlierquery = """
WITH RAW AS(
SELECT BB.season, BB.market, BB.wins, BB.losses, IFNULL(SAFE_DIVIDE(BB.wins,BB.wins+BB.losses),0.5) AS WIN_PCT, AVG(IFNULL(SAFE_DIVIDE(BB.wins,BB.wins+BB.losses),0.5)) OVER (PARTITION BY BB.season) AS MEAN_WIN_PCT,
STDDEV(IFNULL(SAFE_DIVIDE(BB.wins,BB.wins+BB.losses),0.5)) OVER (PARTITION BY BB.season) AS STDDEV_WIN_PCT
FROM `bigquery-public-data.ncaa_basketball.mbb_historical_teams_seasons` BB),
ZS AS (
SELECT *, IFNULL(SAFE_DIVIDE(WIN_PCT-MEAN_WIN_PCT,STDDEV_WIN_PCT),1) AS ZSCORE
FROM RAW)
SELECT *, CASE WHEN ABS(ZSCORE) >= 2 THEN 1 ELSE 0 END AS OUTLIER_FLAG
FROM ZS
ORDER BY season ASC, ABS(ZSCORE) DESC
"""
df = pd.read_gbq(outlierquery)
df.to_csv('C:/Users/jakes/Documents/NCAAMBBWinPctOutliers.csv')